# import

In [1]:
import sys
import os
from tqdm import tqdm

import torch
from torch import nn, optim
from torch.optim import lr_scheduler

import matplotlib.pyplot as plt
import numpy as np

from tensorboardX import SummaryWriter

sys.path.append('../src/')
import utils
import dataloader
from train_utils import *
from model.AutoEncoder import EncoderBlock
from model.RDEC import RDEC

# set params

In [2]:
config = utils.readConfig('../config.json')

In [3]:
train_config = config['train_params']['train']

batch_size = train_config['batch_size']

model_config = config['model_params']
encoder_layers = model_config['encoder_layers']
num_clusters = model_config['num_clusters']

output_dir = '../data/train'
utils.makeDirs(output_dir, ['log', 'weights'])

weight_path = '../data/train/weights/RDEC.pth'

# load dataset

In [4]:
dataloaders, classes = dataloader.load_mnist('../data', batch_size)

# def model

In [5]:
encoder = EncoderBlock(encoder_layers)
for input_data, label in dataloaders['train']:
    break
model = RDEC(encoder, num_clusters, input_data,)
model.load_state_dict(torch.load(weight_path))

<All keys matched successfully>

In [6]:
for imgs, labels in dataloaders['test']:
    break